In [2]:
import copy
import heapq

import numpy as np
from gym import spaces
from gym.utils import seeding

import math
import matplotlib
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from warp_drive.utils.constants import Constants
from warp_drive.utils.data_feed import DataFeed
from warp_drive.utils.gpu_environment_context import CUDAEnvironmentContext

In [3]:
float_dtype = np.float32
int_dtype = np.int32


In [28]:
def base_map2bool_maps(city_base_map, size, height):
    city_maps = np.zeros((size, size, height), dtype=int)

    for i in range(height):
        city_maps[:, :, i] = (city_base_map >= 1)
        city_base_map -= 1
    return city_maps

#### Global state dictionary testing

In [2]:
global_state = {}
# mimic: episode length = 10, num_agents = 10
global_state["key"] = np.zeros((11, 10))
print(global_state["key"].shape[1])

10


In [7]:
value = np.zeros((10,))
assert value.shape[0] == global_state["key"].shape[1]
# global_state["key"][0] = value

In [8]:
global_state["key"][0].shape

(10,)

In [9]:
global_state["key"][0] = value

In [13]:
print(global_state["key"][10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
min_acc = -1
max_acc = 1
num_acc = 10
acc_actions = np.linspace(min_acc, max_acc, num_acc)
print(acc_actions)
acc_actions = np.insert(acc_actions, 0, 0)
print(acc_actions)

[-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ]
[ 0.         -1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111
  0.11111111  0.33333333  0.55555556  0.77777778  1.        ]


### Understanding action spaces

In [19]:
dir_actions = np.array(
    [[0, 0, 1], [0, 0, -1],
     [0, 1, 0], [0, -1, 0],
     [1, 0, 0], [-1, 0, 0]]
).astype(int)
print(dir_actions.shape)
dir_actions = np.insert(dir_actions, 0, [0, 0, 0], axis=0)
print(dir_actions.shape)

(6, 3)
(7, 3)


In [22]:
# from gym import spaces
# print(acc_actions.shape)
# print(dir_actions.shape)
# num_agents = 10
#
# obs_space = {
#     agent_id: spaces.MultiDiscrete(
#         (acc_actions.shape[0], dir_actions.shape[0])
#     )
#     for agent_id in range(num_agents)
# }

(11,)
(7, 3)


In [23]:
# obs_space

{0: MultiDiscrete([11  7]),
 1: MultiDiscrete([11  7]),
 2: MultiDiscrete([11  7]),
 3: MultiDiscrete([11  7]),
 4: MultiDiscrete([11  7]),
 5: MultiDiscrete([11  7]),
 6: MultiDiscrete([11  7]),
 7: MultiDiscrete([11  7]),
 8: MultiDiscrete([11  7]),
 9: MultiDiscrete([11  7])}

### Understanding agents' rewards array

In [ ]:
# agent_type = {}
# num_agents = 10
# num_taggers = 5
# num_runners = 5
# taggers_set = np.random.choice(
#     np.arange(num_agents), num_taggers, replace=False
# )
# taggers = {}
# runners = {}
# for agent_id in range(num_agents):
#     if agent_id in set(taggers_set):
#         agent_type[agent_id] = 1 # taggers
#         taggers[agent_id] = True
#     else:
#         agent_type[agent_id] = 0 # runners
#         runners[agent_id] = True
# print(f"taggers_set: {taggers_set}\n")
# print(f"taggers: {taggers}")
# print(f"runners: {runners}")

In [26]:
# print(f"agent type: {agent_type}")

agent type: {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1}


In [25]:
# step_penalty_for_taggers = -1.0
# step_reward_for_runner = 1.0
#
# step_rewards = [
#     agent_type[agent_id] * step_penalty_for_taggers
#     + (1 - agent_type[agent_id]) * step_reward_for_runner
#     for agent_id in range(num_agents)
# ]
#
# print(f"step rewards: {step_rewards}")

step rewards: [1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0]


## 1. Setting up our own agent type dictionary

In [108]:
num_drones = 10
num_goals = 1
num_obstacles = 10
num_agents = 21

drone_set = np.random.choice(
    np.arange(num_agents - 1), num_drones, replace=False
)

Note that since we always have **1** goal only, we would first randomly choice the index
for drones, and the rest of `num_agent -1` would naturally form the obstacle indices. We would not choose the indices for goal.

In [109]:
agent_type = {}
drones = {}
obstacles = {}
goals = {}
# similarly, only care about the drones and obstacles
for agent_id in range(num_agents - 1):
    if agent_id in set(drone_set):
        agent_type[agent_id] = 1 # drones
        drones[agent_id] = True
    else:
        agent_type[agent_id] = 1 / 2 # obstacles
        obstacles[agent_id] = True

# we append the goal agent id and info at last
agent_type[num_agents-1] = 0 # goal
goals[num_agents] = True

print(f"agent_type: {agent_type}\n")
print(f"drones set: {drone_set}\n")
print(f"drones: {drones}\n")
print(f"obstacles: {obstacles}\n")
print(f"goals:{goals}\n")

agent_type: {0: 0.5, 1: 0.5, 2: 1, 3: 1, 4: 0.5, 5: 0.5, 6: 1, 7: 0.5, 8: 1, 9: 1, 10: 0.5, 11: 0.5, 12: 1, 13: 1, 14: 0.5, 15: 1, 16: 0.5, 17: 1, 18: 0.5, 19: 1, 20: 0}

drones set: [ 3  9 19  2 17  6  8 12 15 13]

drones: {2: True, 3: True, 6: True, 8: True, 9: True, 12: True, 13: True, 15: True, 17: True, 19: True}

obstacles: {0: True, 1: True, 4: True, 5: True, 7: True, 10: True, 11: True, 14: True, 16: True, 18: True}

goals:{21: True}



## 2. Creating unified step rewards array for all agents
There would be only step rewards/penalties for drones and goal, not obstacles.

The example code (`tag_continuous`) uses the key values associated with each agent's type to cleverly calculate corresponding step awards, in a single run of list comprehension.

By utilizing the fact that `step_penalty_for_drone` is always the negative of `step_reward_for_goal`, we can have the reward value for obstacles being **0** by cleverly assigning them **0.5** key values in the `agent_type` dictionary.

And we need to assert this reward property as well.

In [110]:
step_penalty_for_drone = - 1.0
step_reward_for_goal = 1.0
step_rewards = [
    agent_type[agent_id] * step_penalty_for_drone
    + (1 - agent_type[agent_id]) * step_reward_for_goal
    for agent_id in range(num_agents)
]

print(f"step_reward list: {step_rewards}\n")

step_reward list: [0.0, 0.0, -1.0, -1.0, 0.0, 0.0, -1.0, 0.0, -1.0, -1.0, 0.0, 0.0, -1.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 1.0]



## 3. Spawning drones, obstacles and goal given city base map

Here given a base map, which is usually a 10 by 10 numpy array with each entry specifying the height of buildings at that coordinate, we are required to get the coordinates for initializing drones, obstacles, and goal.


### Requirements
- We can easily determine the **empty** position from the base map.
- However, we would like to have more control in spawning the drones and goal.
    - we should be able to determine the relationship between goal and drones at initial state. Such feature would help us configure our environment for different level of difficulties. The relationship includes:
        - whether the drones can detect goal from the start.
        - the distance between drones and goal.
    - the drones would be spawned in a formation, which depends on the number of drones specified, instead of spreading across the entire map.


### ...An intuitive but difficult approach...

We propose the following procedures for initializing the `city`, and spawning `agents`:
1. Generate a `city_base_map` with provided `size` and `difficulty`, or `hint` if provided.
2. Try to spawn the `drones` formation inside the map
    1. If failed, go back to **step 1**, re-generate/modify the map with `hints` to the generator.
    2. Else, proceed to next step.
3. Try to spawn the `goal` with given `difficulty` or other requirements.
    1. If failed, go back to **step 1**, re-generate/modify the map with `hints` to the generator.
    2. Else, proceed to next step.
4. Try to spawn the `obstacles` with given `num_obstacles` in other empty space.
    1. If failed, go back to **step 1**, re-generate/modify the map with `hints` to the generator.
    2. Else, proceed to next step.
5. Return the spawned location of `drones`, `goal`, `obstacles`, and corresponding `city_base_map`.

However, due to the randomness of `city generator`, there is no guarantee that a satisfiable map would be generated, which is the last thing we would like to see. Therefore, we would like the `city generator` modifying an unsatisfiable map to satisfiable, or relax some of the randomness to have greater chance of generating satisfiable map, before re-generating a completely random new one which might fail the trials.

To facilitate this, `hints` would be provided to the generator....*(but really, how to put down this ambiguous `hints` into mathematics form, then implement them in codes?)*


### ...A not so intuitive approach but solve the problem with minimal workload
#### *....why not we spawn the drones, goal, and obstacles before generating buildings?*

We would modify our `city generator` to accept following arguments:
1. `num_drones`: number of drones need to spawn, and pack within a formation.
2. `obs_range`: the range of observation for drones.
3. `if_within_obs`: whether to spawn the goal within drones' formation's observation range.
4. `if_directly_obs`: whether the `goal` is directly observable to the `drone`. If `True`, there should not be buildings or obstacles in the line of sight. Else, the line of sight is interrupted.
5. `distance`: the distance between centroid of drones' formation and goal. This would be a multiplier on top of environment size, thus should fall between 0 and 1.
6. `num_obstacles`: the number of obstacles to spawn randomly.7
7. `difficulty`: the difficulty index related to cluster-ness of the environment.

The new, with spawning-awareness, `city generator` would follow this procedure instead:
1. Designate a cube/space for the `drones` formation. Calculate the centroid coordinate and drones' coordinates.
2. Designate a cube for `goal`. Calculate goal coordinate.
3. Designate several cube for `obstacles`. Calculate their coordinates.
4. Collect the above `designated cube`s.
    - `designated cube` would invalidate some building placements. `cubes` placed above the ground would imply *no buildings* in this block or *building in this block lower than* `cube`.
    - all `designated cube` should be placed above a minimum height from ground. This is to avoid the conflict with `fill_per_sec` requirement.
    - the `drones` formation would always be spawned from the same centroid locations, and the `goal` location would always be in front of the `drones`.
5. Generate the city without touching the `designated cells`, and conform to `if_directly_obs` requirement. Procedures are as:
    - Generate the city without considering the `designated cells`.
    - Calculate the updated `fill_per_sec`, and `mean_height`, `std_height` parameters after first generation.
    - Compared these 3 parameters with prescribed inputs.
    - Then first check `designated cells` of `drones` and `goal`. Randomly select either to remove the entire building or lower its height based on RNG.
    - Check if buildings conform to `is_directly_obs` requirement (i.e., check the *line of sight* between centroid of `drones` formation and `goal`).
        - If there are buildings blocking line of sight,




In [17]:
import copy
import heapq

import numpy as np
from gym import spaces
from gym.utils import seeding

import math
import matplotlib
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from warp_drive.utils.constants import Constants
from warp_drive.utils.data_feed import DataFeed
from warp_drive.utils.gpu_environment_context import CUDAEnvironmentContext
float_dtype = np.float32
int_dtype = np.int32

In [5]:
def Bresenham3D(x1, y1, z1, x2, y2, z2):
    ListOfPoints = [[x1, y1, z1]]
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    if x2 > x1:
        xs = 1
    else:
        xs = -1
    if y2 > y1:
        ys = 1
    else:
        ys = -1
    if z2 > z1:
        zs = 1
    else:
        zs = -1

    # Driving axis is X-axis"
    if dx >= dy and dx >= dz:
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while x1 != x2:
            x1 += xs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dx
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz
            ListOfPoints.append([x1, y1, z1])

    # Driving axis is Y-axis"
    elif dy >= dx and dy >= dz:
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while y1 != y2:
            y1 += ys
            if p1 >= 0:
                x1 += xs
                p1 -= 2 * dy
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz
            ListOfPoints.append([x1, y1, z1])

    # Driving axis is Z-axis"
    else:
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while z1 != z2:
            z1 += zs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dz
            if p2 >= 0:
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx
            ListOfPoints.append([x1, y1, z1])
    return ListOfPoints

In [6]:
def spawn_and_generate_city(
        size,
        max_height,
        difficulty_level,
        obs_range,
        is_within_obs,
        is_directly_obs,
        distance,
        agent_type,
        num_drones,
        num_obstacles,
        num_goals=1,
        sec_num=2):
    
    city_base_map = np.zeros((size, size), dtype=int)
    env_size = 10
    # the interpolation of mean height and std height
    mean_height_list = np.linspace(0.3, 0.9, 10)
    std_height_list = np.geomspace(2, 0.4, 10)
    
    # randomly sample pair from above
    idx = random.randint(0, len(mean_height_list)-1)
    mean_height = mean_height_list[idx]
    std_height = std_height_list[idx]
    
    # based on difficulty level, select the cluster-ness
    fill_per_sec = difficulty_level / 10
    
    h_mean = int(max_height * mean_height)
    h_std = int(h_mean * std_height)
    cell_size = int(size / sec_num)
    num = int(fill_per_sec * cell_size**2)

    a_min = np.zeros(num, dtype=int)
    a_max = np.full(num, max_height, dtype=int)

    # spawn the drones formation around the centroid of block (5, 0, 5), using default
    drones_loc_dict = spawn_drones(num_drones, agent_type)

    # spawn the goal location, using default block centroid of (5, 0, 5)
    # the distance here is a floating point value.
    distance = distance * env_size
    goal_loc_dict = spawn_goal(distance, agent_type)
    goal_loc = goal_loc_dict[num_agents - 1]

    obstacles_loc_dict = spawn_obstacles(num_obstacles, agent_type, goal_loc)

    # convert the float coordinates into block ids
    drones_blocks_id = get_blocks_id_list(drones_loc_dict)
    obstacles_blocks_id = get_blocks_id_list(obstacles_loc_dict)
    goal_blocks_id = get_blocks_id_list(goal_loc_dict)

    # get the blocks that in the line of sight between drones and goal.
    # make use of Bresenham3D algorithm, and we only care about the drone centroid.
    # drone centroid should be a class attribute
    drones_centroid = np.array([5, 0, 5]).astype(int_dtype)
    gx, gy, gz = goal_loc[0]
    d_x, d_y, d_z = drones_centroid
    blk_sight_list = Bresenham3D(gx, gy, gz, d_x, d_y, d_z)

    occupied_blocks_list = np.concatenate(
        drones_blocks_id, obstacles_blocks_id, goal_blocks_id
    )

    # first just generate
    for i in range(sec_num):
        for j in range(sec_num):
            building_height = np.clip(np.random.normal(h_mean, h_std, num), a_min, a_max)
            height_list = np.concatenate((building_height, np.zeros(cell_size*cell_size - num)))
            np.random.shuffle(height_list)
            block_height = np.reshape(height_list, (cell_size, cell_size))
            city_base_map[i*cell_size: (i+1)*cell_size, j*cell_size:(j+1)*cell_size] = block_height
            
    # sum_height = city_base_map.sum()
    # fill_ratio = sum_height / (size * size * max_height)
    # specific_fill_ratio = sum_height / (num * sec_num * sec_num * max_height)

    # TODO: how many blocks must be removed/added at each locs - hard quotas map
    # check the occupied location

    # TODO: how many blocks can be validly removed/added at each locs - soft quotas maps (one add map, one remove map)
    # TODO: a range around the mean and std variance - satisfiable

    # TODO: convert hard quotas into soft quotas by executing necessary changes first - no hard quotas
    # TODO: update soft quotas map

    # TODO: Loops: if there are still soft quotas and not satisfiable:
    # TODO:     check current fill_per_sec versus input values - hints

    # TODO:     if fill_per_sec is lower than expected: move twd add, else move twd remove (a        stochastic strategy)

    # TODO:         check local quotas, if available, only add or remove 1 block locally.
    # TODO:         update soft quotas maps

    # TODO:     if passed certain updates but not yet satisfiable, relax requirements.

    # TODO:     check if the map is satisfiable.




    return city_base_map

In [7]:
def get_blocks_id_list(loc_dict):
    """
    Given a location dictionary whose values are float coordinates,
    get the blocks ids these coordinates are in.
    :param loc_dict:
    :return:
    """
    loc_list = np.unique(
        [
            (coords - coords % 1).astype(int_dtype) for coords in loc_dict.values()
        ],
        axis=0
    )
    return loc_list

In [8]:
def spawn_goal(distance, agent_type, drone_centroid=np.array([5, 0, 5])):
    """
    can be replaced with (self).
    only accept 1 goal. distance and agent type can be accessed via self.
    :param distance: a floating number indicating distance between.
    :param agent_type:
    :param centroid:
    :return: a goal dict whose key is goal agent_type id, and a goal coord list.
    """
    # goal would be randomly placed within this sphere around the centroid

    # get the goal agent_ids
    goal_id = [i for i in range(num_agents) if agent_type[i] == 0]
    assert len(goal_id)==1
    goal_dict = {}
    distance = np.array([0, 1, 0]) * distance
    place_radius = 0.4
    displacement = np.random.random(3) * place_radius
    # print(f"displacement: {displacement}")
    goal_dict[goal_id[0]] = drone_centroid + distance + displacement
    return goal_dict

In [9]:
def spawn_drones(num_drones, agent_type, centroid=np.array([5, 0, 5])):
    """
    can be replaced with (self).
    num_drones and agent_type can be accessed via self.
    :return: a drones dict whose keys are drone agent type id, and a drones coord list.
    """
    # spacing parameters, determine the initial distance between drones
    spacing = float_dtype(0.1)
    # actually, the coordinate in centroid is indicating the block number
    # we need to stagger it to get the block centroid coords
    stagger = np.array([0.5, 0, 0.5]).astype(float_dtype)
    # print(centroid)
    centroid = centroid + stagger
    even_rule = np.array([
        [-1, 0, 0], [1, 0, 0],
        [0, 0, -1], [0, 0, 1],
        [-1, 0, 1], [1, 0, -1],
        [1, 0, 1], [-1, 0, -1]
    ], dtype=int_dtype)

    # get the drone id
    drones_id = [i for i in range(num_agents) if agent_type[i] == 1]
    assert len(drones_id)==num_drones

    drones_loc = {}
    if num_drones == 1:
        drones_loc[drones_id[0]] = centroid
    elif num_drones % 2 == 0: # even number of drones
        for _id in range(num_drones):
            # print("step:", even_rule[drone_id % len(even_rule)])
            # print("spacing:", spacing)
            # print("iter:", math.ceil(drone_id / len(even_rule)))
            update = even_rule[_id % len(even_rule)] * spacing * math.ceil((_id + 1) / len(even_rule))
            # print("update: ", update, "type: ", type(update))
            drones_loc[drones_id[_id]] = (
                centroid + update
            )
    else:
        # odd number of drones
        drones_loc[drones_id[0]] = centroid
        for _id in range(num_drones - 1):
            update = even_rule[_id % len(even_rule)] * spacing * math.ceil((_id
              + 1)/ len(even_rule))
            drones_loc[drones_id[_id + 1]] = (
                centroid + update
            )

    return drones_loc

In [23]:
def spawn_obstacles(num_obstacles, agent_type, goal_loc_centroid, drones_loc_centroid=np.array([5.5, 0, 5.5])):
    """

    :param num_obstacles:
    :param agent_type:
    :param goal_loc_centroid:
    :param drones_loc_centroid:
    :return: a obstacles dict whose keys are obstacles agent type id, and a obstacles coord list
    """
    # get the occupied blocks of drones and goal
    # 3 by 3 blocks around drones centroid and 3 by 3 blocks around goal centroid are reserved.
    env_size = 10
    env_max_height = 20
    env_min_height = 5 # no spawning of obstacles below this height
    stagger = np.array([0.5, 0, 0.5]).astype(float_dtype)
    place_radius = 0.4
    displacement = (np.random.random(3) * place_radius).astype(float_dtype)
    # get the obstacles agent ids
    obstacles_id = [i for i in range(num_agents) if agent_type[i] == 0.5]
    assert len(obstacles_id)==num_obstacles
    # drones_loc_centroid += stagger
    # goal_loc_centroid += stagger
    three_by_three_walk = np.array(
        [
            [0, 0, 0], [1, 0, 0], [-1, 0, 0],
            [0, 0, 1], [1, 0, 1], [-1, 0, 1],
            [0, 0, -1], [1, 0, -1], [-1, 0, -1]
        ]
    ).astype(int_dtype)
    drones_blocks_centroids = [(
            drones_loc_centroid + three_by_three_walk[i])
            for i in range(three_by_three_walk.shape[0]) ]

    goal_blocks_centroids = [(
            goal_loc_centroid + three_by_three_walk[i])
            for i in range(three_by_three_walk.shape[0]) ]

    occupied = np.concatenate((
        drones_blocks_centroids, goal_blocks_centroids))

    empty_blocks_id = [
        np.array([x, y, z]).astype(int_dtype)
        for x in range(env_size)
        for y in range(env_size)
        for z in range(env_min_height, env_max_height) if not
        np.any(
            np.all(
                np.array([x, y, z]).astype(int_dtype) == occupied, axis=1
            )
        )
    ]

    empty_indices = np.random.choice(
        len(empty_blocks_id), num_obstacles, replace=False
    )

    obstacles_blocks = [
        empty_blocks_id[empty_indices[i]] for i in range(num_obstacles)
    ]

    obstacles_loc_dict = {
        obstacles_id[_id]: obstacles_blocks[_id]
        for _id in range(num_obstacles)
    }
     # obstacles_coord_list = [
     #     obstacles_blocks[i] for i in range(num_obstacles)
     # ]
    return obstacles_loc_dict


In [11]:
def base_map2bool_maps(city_base_map, size, height):
    """
    Return the bool maps of the city for 3-D rendering.
    :param city_base_map:
    :param size:
    :param height:
    :return:
    """
    city_maps = np.zeros((size, size, height), dtype=int)

    for i in range(height):
        city_maps[:, :, i] = (city_base_map >= 1)
        city_base_map -= 1
    return city_maps

In [12]:
def test_generate_city(size, max_height, difficulty_level, sec_num=2):

    city_base_map = np.zeros((size, size), dtype=int)

    # the interpolation of mean height and std height
    mean_height_list = np.linspace(0.3, 0.9, 10)
    std_height_list = np.geomspace(2, 0.4, 10)

    # randomly sample pair from above
    idx = random.randint(0, len(mean_height_list)-1)
    mean_height = mean_height_list[idx]
    std_height = std_height_list[idx]

    # based on difficulty level, select the cluster-ness
    fill_per_sec = difficulty_level / 10

    h_mean = int(max_height * mean_height)
    h_std = int(h_mean * std_height)
    cell_size = int(size / sec_num)
    num = int(fill_per_sec * cell_size**2)

    a_min = np.zeros((num), dtype=int)
    a_max = np.full((num), max_height, dtype=int)

    for i in range(sec_num):
        for j in range(sec_num):
            building_height = np.clip(np.random.normal(h_mean, h_std, num), a_min, a_max)
            height_list = np.concatenate((building_height, np.zeros(cell_size*cell_size - num)))
            np.random.shuffle(height_list)
            block_height = np.reshape(height_list, (cell_size, cell_size))
            city_base_map[i*cell_size: (i+1)*cell_size, j*cell_size:(j+1)*cell_size] = block_height

    sum_height = city_base_map.sum()
#     fill_ratio = sum_height / (size * size * max_height)
#     specific_fill_ratio = sum_height / (num * sec_num * sec_num * max_height)

    return city_base_map

In [13]:
def get_buildings_dict(cbm):
    env_size = 10
    b_dict = {}
    for x in range(env_size):
        for y in range(env_size):
            if cbm[x, y]:
                b_dict[(x, y)] = cbm[x, y]
    return b_dict

In [42]:
import mayavi.mlab
from mayavi import mlab
# mlab.init_notebook()
def mayavi_render_point(p_):
    x_, y_, z_ = np.transpose(p_)
    mayavi.mlab.points3d(
        x_, y_, z_, color=(1, 0, 0), scale_factor=0.05
    )
    mayavi.mlab.show()

def mayavi_render_city(data):
    xx, yy, zz = np.where(data==1)
    mayavi.mlab.points3d(
        xx, yy, zz, mode="cube", color=(0, 1, 0), scale_factor=1
    )
    mayavi.mlab.show()

def mayavi_render_all(data, p_):
    """
    render city, as well as drones, goal, and obstacles with different colors.
    :param data: city bool maps
    :param p_: expected three un-transposed coords lists
    :return: none.
    """
    mlab.barchart(data)
    mlab.vectorbar()

    a_, b_, c_ = p_

    x1, y1, z1 = np.transpose(a_)
    mayavi.mlab.points3d(
        x1, y1, z1, color=(1, 0, 0), scale_factor=1
    )

    x2, y2, z2 = np.transpose(b_)
    mayavi.mlab.points3d(
        x2, y2, z2, color=(0.25, 0.25, 0.5), scale_factor=1
    )

    x3, y3, z3 = np.transpose(b_)
    mayavi.mlab.points3d(
        x3, y3, z3, color=(0.25, 0.5, 0.25), scale_factor=1
    )
    mayavi.mlab.show()


# data = (100, 100, 100)
# data = np.zeros(data)
# data[0:50, 50:70, 0:50] = 1
# data[0:50, 0:20, 0:50] = 1
#
# xx, yy, zz = np.where(data == 1)
#
# mayavi.mlab.points3d(xx, yy, zz,
#                      mode="cube",
#                      color=(0, 1, 0),
#                      scale_factor=1)
# mayavi.mlab.show()

## Integrated testing

#### Spawn the stuffs individually.

In [24]:
num_drones = 10
num_goals = 1
num_obstacles = 10
num_agents = 21

drone_set = np.random.choice(
    np.arange(num_agents - 1), num_drones, replace=False
)
agent_type = {}
drones = {}
obstacles = {}
goals = {}
# similarly, only care about the drones and obstacles
for agent_id in range(num_agents - 1):
    if agent_id in set(drone_set):
        agent_type[agent_id] = 1 # drones
        drones[agent_id] = True
    else:
        agent_type[agent_id] = 1 / 2 # obstacles
        obstacles[agent_id] = True

# we append the goal agent id and info at last
agent_type[num_agents-1] = 0 # goal
goals[num_agents] = True

print(f"agent_type: {agent_type}\n")
print(f"drones set: {drone_set}\n")
print(f"drones: {drones}\n")
print(f"obstacles: {obstacles}\n")
print(f"goals:{goals}\n")


agent_type: {0: 0.5, 1: 1, 2: 0.5, 3: 1, 4: 0.5, 5: 1, 6: 0.5, 7: 1, 8: 1, 9: 0.5, 10: 0.5, 11: 0.5, 12: 0.5, 13: 0.5, 14: 1, 15: 1, 16: 1, 17: 1, 18: 0.5, 19: 1, 20: 0}

drones set: [ 5  8  3 17  1 19 15  7 14 16]

drones: {1: True, 3: True, 5: True, 7: True, 8: True, 14: True, 15: True, 16: True, 17: True, 19: True}

obstacles: {0: True, 2: True, 4: True, 6: True, 9: True, 10: True, 11: True, 12: True, 13: True, 18: True}

goals:{21: True}



In [25]:
o_dict = spawn_obstacles(10, agent_type, np.array([5, 3, 5]))
d_dict = spawn_drones(10, agent_type)
d_blk_centroid = np.array([5, 0, 5]).astype(int_dtype)
dis = 2.7
g_dict = spawn_goal(dis, agent_type)

o_list = get_blocks_id_list(o_dict)
d_list = get_blocks_id_list(d_dict)
g_list = get_blocks_id_list(g_dict)

# print(o_list)
# print(len(o_list))
# print(d_list)
# print(len(d_list))
# print(g_list)

occ_list = np.concatenate((o_list, d_list, g_list))
gx, gy, gz = g_list[0]
d_centroid = np.array([5, 0, 5]).astype(int_dtype)
d_x, d_y, d_z = d_centroid
path = Bresenham3D(gx, gy, gz, d_x, d_y, d_z)
print(d_dict)
# mayavi_render_point(d_list)

{1: array([5.4, 0. , 5.5]), 3: array([5.6, 0. , 5.5]), 5: array([5.5, 0. , 5.4]), 7: array([5.5, 0. , 5.6]), 8: array([5.4, 0. , 5.6]), 14: array([5.6, 0. , 5.4]), 15: array([5.6, 0. , 5.6]), 16: array([5.4, 0. , 5.4]), 17: array([5.3, 0. , 5.5]), 19: array([5.7, 0. , 5.5])}


In [26]:
# print(occ_list)
print(o_list)

[[ 1  3 18]
 [ 1  8 18]
 [ 2  1 14]
 [ 3  6 12]
 [ 3  6 19]
 [ 3  8 10]
 [ 4  0 13]
 [ 5  1 12]
 [ 6  9  8]
 [ 9  3 12]]


In [27]:
d_coords = np.array([coord for coord in d_dict.values()])
# print(d_coords)
d_ = np.transpose(d_coords)
# print(d_)
mayavi_render_point(d_coords)

#### Check and execute hard quota

In [28]:
c_b_m = test_generate_city(10, 20, 5)
print(c_b_m)
path_clr = True

h_q_m = np.zeros((10, 10), dtype=int_dtype)
for (x, y, z) in occ_list:
    x_ = x
    y_ = y
    print(f"occ: checking loc:{x, y}, occ: {z}, have: {c_b_m[x_, y_]}")
    h_q_m[x_, y_] = \
        (z - (c_b_m[x_, y_] + 1)) * (c_b_m[x_, y_] + 1 - z > 0)

# path clr for clear line of sight
# first check the removed, denoted as negative entries in hard quota
# building height < occ height
for (x, y, z) in path:
    x_ = x
    y_ = y
    print(f"path: checking loc:{x, y}, have: {c_b_m[x_, y_]}")
    if path_clr:
        h_q_m[x_, y_] = \
            (5 - (c_b_m[x_, y_] + 1)) * (c_b_m[x_, y_] + 1 > 5)
    else:
        h_q_m[x_, y_] = (5 - (c_b_m[x_, y_])) * (c_b_m[x_, y_] < 5)
print(h_q_m)

[[19  0  0 12 13 10  0  9  0 20]
 [ 0 19  0  0  5  0  0 15  0  0]
 [20 16  0  0  0  9 20  0  7  0]
 [ 0  8 15  7 16  0  9  0 20 20]
 [ 0  0 14  0  0 14  0  0  9  0]
 [14 12  0  0  0  0  0  0  9  0]
 [ 0 20  0  8  0 19  0  0 20  0]
 [ 0 15 16  0  0 16 20  0  0  0]
 [ 0 20  4  0 20  8  0 20 13 16]
 [15  8  0  0 12 20  0  0 20 20]]
occ: checking loc:(1, 3), occ: 18, have: 0
occ: checking loc:(1, 8), occ: 18, have: 0
occ: checking loc:(2, 1), occ: 14, have: 16
occ: checking loc:(3, 6), occ: 12, have: 9
occ: checking loc:(3, 6), occ: 19, have: 9
occ: checking loc:(3, 8), occ: 10, have: 20
occ: checking loc:(4, 0), occ: 13, have: 0
occ: checking loc:(5, 1), occ: 12, have: 12
occ: checking loc:(6, 9), occ: 8, have: 0
occ: checking loc:(9, 3), occ: 12, have: 0
occ: checking loc:(5, 0), occ: 5, have: 14
occ: checking loc:(5, 3), occ: 5, have: 0
path: checking loc:(5, 3), have: 0
path: checking loc:(5, 2), have: 0
path: checking loc:(5, 1), have: 12
path: checking loc:(5, 0), have: 14
[[  0   0 

In [40]:
# apply the changes
c_b_m_hard = c_b_m + h_q_m
print(c_b_m_hard)
mlab.barchart(c_b_m_hard)
mlab.vectorbar()
mlab.show()

[[19  0  0 12 13 10  0  9  0 20]
 [ 0 19  0  0  5  0  0 15  0  0]
 [20 13  0  0  0  9 20  0  7  0]
 [ 0  8 15  7 16  0  9  0  9 20]
 [ 0  0 14  0  0 14  0  0  9  0]
 [ 4  4  0  0  0  0  0  0  9  0]
 [ 0 20  0  8  0 19  0  0 20  0]
 [ 0 15 16  0  0 16 20  0  0  0]
 [ 0 20  4  0 20  8  0 20 13 16]
 [15  8  0  0 12 20  0  0 20 20]]


In [43]:
# test and visualize
d_coords = np.array([coord for coord in d_dict.values()])
o_coords = np.array([coord for coord in o_dict.values()])
g_coords = np.array([coord for coord in g_dict.values()])

# c_bool_m_hard = base_map2bool_maps(c_b_m_hard, 10, 20)

mayavi_render_all(c_b_m_hard, (d_coords,o_coords,g_coords))

/home/hibiki/miniconda3/envs/warp_drive/lib/python3.7/site-packages/traitsui/__init__.py:23: DeprecationWarning: traitsui.__version__ is deprecated, use impportlib.metadata.version('traitsui') 
  DeprecationWarning,
libpng warning: iCCP: known incorrect sRGB profile
/home/hibiki/miniconda3/envs/warp_drive/lib/python3.7/site-packages/traitsui/qt/ui_panel.py:1360: DeprecationWarning: setting create=False is no longer required
  widget = HeadingText(*args, create=create, **kw)
2023-06-04 20:33:09.866 (1818.536s) [        4BE9D740]vtkLabeledDataMapper.cx:454   WARN| vtkLabeledDataMapper (0x559319888430): Could not find label array (index 0) in input.
2023-06-04 20:33:11.630 (1820.299s) [        4BE9D740]vtkLabeledDataMapper.cx:454   WARN| vtkLabeledDataMapper (0x559319888430): Could not find label array (index 0) in input.
2023-06-04 20:33:11.688 (1820.358s) [        4BE9D740]vtkLabeledDataMapper.cx:454   WARN| vtkLabeledDataMapper (0x559319888430): Could not find label array (index 0) in 

In [44]:
import numpy as np
import mayavi.mlab as mlab
from numpy import exp, sin, cos, tan, random, mgrid, ogrid, linspace, sqrt, pi
import matplotlib.pyplot as plt
mlab.figure(fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))

def peak(x, y):
    return 3.0*(1.0 - x)**2*exp(-(x**2) - (y + 1.0)**2) - 10*(x/5.0 - x**3 - y**5) * exp(- x**2 - y**2 ) - 1.0 / 3.0 * exp(-(x+1.0)**2 - y**2)

# s = np.random.rand(3, 3)
# mlab.barchart(s)
# mlab.vectorbar()
# mlab.show()

x, y = np.mgrid[0.5:8.5:9j, 0.5:8.5:9j]
# s = peak(x, y)
# mlab.barchart(x, y, c_b_m_hard)
# mlab.vectorbar()
# mlab.show()

AssertionError: argument shape are not equal

In [1]:
import heapq
max_iter = 50

In [4]:

def shortest_path_dijkstra_matrix(adjacency_matrix, start, target):
    num_nodes = len(adjacency_matrix)
    distances = [float('inf')] * num_nodes
    distances[start] = 0
    queue = [(0, start)]
    parent = {}
    visited = []
    iter_num = 0
    while queue and iter_num < max_iter:
        print(f"curr queue: {queue}")
        current_distance, current_node = heapq.heappop(queue)
        print(f"pop: curr dis: {current_distance}, curr node: {current_node}")
        if current_node == target:
            path = [current_node]
            while current_node != start:
                current_node = parent[current_node]
                path.append(current_node)
            print(f"distances list: {distances}")
            return path[::-1]

        if current_distance > distances[current_node]:
            continue

        for neighbor in range(num_nodes):
            weight = adjacency_matrix[current_node][neighbor]
            if weight > 0:
                print(f"checking: {neighbor}, weight: {weight}")
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    print(f"updating: distance: {distance}, neighbor: {neighbor}")
                    heapq.heappush(queue, (distance, neighbor))
                    parent[neighbor] = current_node
        iter_num += 1
        print(iter_num)
    return None

In [5]:
import numpy as np
adj_matrx = np.array(
    [[-1, 3, 1, -1],
     [3, -1, -1, -1],
     [1, -1, -1, 2],
     [-1, -1, 2, -1]]
)
start = 0
end = 3
path = shortest_path_dijkstra_matrix(adj_matrx, start, end)

curr queue: [(0, 0)]
pop: curr dis: 0, curr node: 0
checking: 1, weight: 3
updating: distance: 3, neighbor: 1
checking: 2, weight: 1
updating: distance: 1, neighbor: 2
1
curr queue: [(1, 2), (3, 1)]
pop: curr dis: 1, curr node: 2
checking: 0, weight: 1
checking: 3, weight: 2
updating: distance: 3, neighbor: 3
2
curr queue: [(3, 1), (3, 3)]
pop: curr dis: 3, curr node: 1
checking: 0, weight: 3
3
curr queue: [(3, 3)]
pop: curr dis: 3, curr node: 3
distances list: [0, 3, 1, 3]
